In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import namedtuple
import os
from time import time
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision
import torchvision.transforms as transforms

import opacus
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator

from utils import generate_run_id, get_input_args, Args
from models import Discriminator, Generator_MNIST, Weight_Clipper, G_weights_init
from data import load_MNIST
from metrics import get_IS, get_FID

In [ ]:
# Random Seeding
torch.manual_seed(0)
np.random.seed(0)

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

activation = 'LeakyReLU'
args = Args(
    # Model Parameters
    hidden=[16, 12], nz=100, ngf=32, nc=1, activation=activation,
    # Privacy Parameters
    epsilon=50.0, delta=1e-6, noise_multiplier=0.3, c_p=0.01, 
    # Training Parameters
    lr=1e-3, beta1=0.5, batch_size=16, n_d=3, n_g=int(1e4)
)

# Generate Run ID
run_id = generate_run_id(args)


run_id = "public_16-12_100_32_1_inf_1e-06_0.4_0.005_0.0001_0.5_64_4_300000_LeakyReLU"
# run_id = "16-12_100_32_1_inf_1e-06_0.4_0.005_0.0005_0.5_64_5_50000"
# /home/jason/p2/runs/16-12_100_32_1_inf_1e-06_0.4_0.005_0.0005_0.5_64_5_50000
# run_id = "16-12_100_32_1_50.0_1e-06_0.6_0.005_0.0001_0.5_64_4_300000_Tanh"
# run_id = "16-12_100_32_1_50.0_1e-06_0.6_0.005_0.0001_0.5_64_5_300000_Tanh"
run_id = "16-12_100_32_1_50.0_1e-06_0.2_0.005_0.0001_0.5_64_4_300000_Tanh"

# Create Folder Path
run_fp = os.path.join('runs/', run_id)
run_fp

In [ ]:
import os
# run_id = "16-12_100_32_1_50.0_1e-06_0.6_0.005_0.0001_0.5_32_4_500000"
run_fp = os.path.join('runs/', run_id)

# Read loss.txt
# 4 lines of discriminator loss, 1 line of generator loss
# Seperate discriminator loss and generator loss
loss_fp = os.path.join(run_fp, 'loss.txt')
epsilons = []
d_loss, g_loss = [], []

with open(loss_fp, 'r') as f:
    loss = f.read().splitlines()
    for i in range(len(loss)):
        if "time" in loss[i]:
            continue
        if i % 5 == 4:
            idx, l = loss[i].split(", ")
            idx = int(float(idx))
            g_loss.append((idx, float(l)))
            # print("G", loss[i])
        else:
            idx, l = loss[i].split(", ")
            idx = int(float(idx))
            d_loss.append((idx, float(l)))
            # print(loss[i])
        
        # if "epsilon" in loss[i]:
        #     idx, epsilon = loss[i].split(": epsilon")
        #     epsilons.append(float(epsilon))
            # print(idx, epsilon)

# print difference between epsilons
# for i in range(len(epsilons)-1):
#     print(epsilons[i+1] - epsilons[i])

# print difference of difference between epsilons
# for i in range(len(epsilons)-2):
#     print(epsilons[i+2] - epsilons[i+1] - epsilons[i+1] + epsilons[i])



In [ ]:
g_loss[197: 201]

In [ ]:
# Graph Loss
plt.figure(figsize=(10, 5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(*zip(*d_loss), label="Discriminator")
plt.plot(*zip(*g_loss), label="Generator")
plt.xlabel("iterations")
plt.ylabel("Loss")

# Only show first 100 epochs
# plt.xlim(-10, 20000)
plt.legend()
plt.show()

In [ ]:
# Load Accounant
accountant_fp = os.path.join(run_fp, 'accountant_2000.pth')
accountant = torch.load(accountant_fp)

# accountant.get_epsilon(1e-5)


In [ ]:
# Setup MNIST dataset
labeling_loader, public_loader, private_loader, test_loader = load_MNIST(args.batch_size)

In [ ]:

def generate_samples(fp):
    G = Generator_MNIST(nz=args.nz, ngf=args.ngf, nc=args.nc).to(device)
    G.load_state_dict(torch.load(os.path.join(run_fp, fp)))
    G.eval()
    # Generate Sample Images
    noise = torch.randn(32, 100, 1, 1).to(device)
    fake = G(noise)
    fake = fake.view(fake.size(0), 1, 28, 28)
    print(torch.min(fake), torch.max(fake))

    # Plot Sample Images
    fig, ax = plt.subplots(4, 8, figsize=(10, 5))
    for i in range(4):
        for j in range(8):
            ax[i, j].imshow(fake[i*8+j][0].detach().cpu().numpy(), cmap='gray')
            ax[i, j].axis('off')
    plt.show()

In [ ]:
# Load the generator
run_id = "16-12_100_32_1_50.0_1e-06_0.1_0.01_0.0001_0.5_64_10_300000_Tanh"
run_id = "runs/private_16-12_100_32_1_0_1e-06_0.0_0.01_0.0001_0.5_64_5_200000_LeakyReLU"
run_id = run_id.split("/")[-1]
# run_id = "private_16-12_100_32_1_inf_1e-06_0.4_0.005_0.0001_0.5_64_4_300000_LeakyReLU"
run_fp = os.path.join('runs/', run_id)

# for i in [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000, 4200, 4400, 4600, 4800, 5000]:
#     fp = 'netG_{}.pt'.format(i)
#     if os.path.exists(os.path.join(run_fp, fp)):
#         print(fp)
#         generate_samples(fp)
# for i in [5200, 5400, 5600, 5800, 6000, 6200, 6400, 6600, 6800, 7000, 7200, 7400, 7600, 7800, 8000, 8200, 8400, 8600, 8800, 9000, 9200, 9400, 9600, 9800, 10000, 10200, 10400, 10600, 10800, 11000, 11200, 11400, 11600, 11800, 12000, 12200, 12400, 12600, 12800, 13000, 13200, 13400, 13600, 13800, 14000, 14200, 14400, 14600, 14800, 15000, 15200, 15400, 15600, 15800, 16000, 16200, 16400, 16600, 16800, 17000, 17200, 17400, 17600, 17800, 18000, 18200, 18400, 18600, 18800, 19000, 19200, 19400, 19600, 19800, 20000, 20200, 20400, 20600, 20800, 21000, 21200, 21400, 21600, 21800, 22000, 22200, 22400, 22600, 22800, 23000, 23200, 23400, 23600, 23800, 24000, 24200, 24400, 24600, 24800, ]:
#     fp = 'netG_{}.pt'.format(i)
#     if os.path.exists(os.path.join(run_fp, fp)):
#         print(fp)
#         generate_samples(fp)
for i in [25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 135000, 140000, 145000, 150000, 155000, 160000, 165000, 170000, 175000, 180000, 185000, 190000, 195000, 200000, 205000, 210000, 215000, 220000, 225000, 230000, 235000, 240000, 245000, 250000, 255000, 260000, 265000, 270000, 275000, 280000, 285000, 290000, 295000, 300000]:
    fp = 'netG_{}.pt'.format(i)
    if os.path.exists(os.path.join(run_fp, fp)):
        print(fp)
        generate_samples(fp)

In [ ]:
# Generate 2048 fake images
noise = torch.randn(2048, 100, 1, 1).to(device)
fake = G(noise)
fake = fake.view(fake.size(0), 1, 28, 28)

In [ ]:
# Calculate Inception Score
IS = get_IS(fake)
print("Inception Score:", IS)

In [ ]:
# Calculate Frechet Inception Distance
FID = get_FID(fake)
print("Frechet Inception Distance:", FID)